In [1]:
!pip install pytorch-pretrained-bert pytorch-nlp
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForSequenceClassification

     |████████████████████████████████| 133kB 5.8MB/s 
     |████████████████████████████████| 92kB 8.4MB/s 
     |████████████████████████████████| 133kB 45.8MB/s 
     |████████████████████████████████| 71kB 7.8MB/s 
     |████████████████████████████████| 7.1MB 31.1MB/s 
ERROR: botocore 1.19.36 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [6]:
import pickle

loader = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/mindhug/pickled model/loader.pkl', 'rb'))
paddding = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/mindhug/pickled model/pad_seqience.pkl', 'rb'))
sampler = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/mindhug/pickled model/sampler.pkl', 'rb'))
tensor_data= pickle.load(open('/content/drive/MyDrive/Colab Notebooks/mindhug/pickled model/tensor_data.pkl', 'rb'))


In [7]:
label_dict={'anger': 2,
 'disgust': 4,
 'fear': 1,
 'guilt': 6,
 'happiness': 7,
 'joy': 0,
 'sadness': 3,
 'shame': 5}

In [5]:
# load the model from disk
import torch
if torch.cuda.is_available():
    map_location=lambda storage, loc: storage.cuda()
else:
    map_location='cpu'
    

bert_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=9)
bert_model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/mindhug/pickled model/bert_model (1).pt',map_location=map_location))





<All keys matched successfully>

In [8]:
tokenizer = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/mindhug/pickled model/bert_tokenizer.pkl', 'rb'))

In [ ]:

import pandas as pd
# Load the dataset into a pandas dataframe.
input_text=input('Enter the text: ')
new_sentence=pd.Series(input_text)

# Report the number of sentences.
#print('Number of test sentences: {:,}\n'.format(df.shape[0]))
# Create sentence and label lists
sentences = new_sentence.values
# tokenize test data
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
MAX_LEN = 128
# Pad our input tokens
input_ids = paddding([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = paddding(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Create attention masks
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 

# create test tensors
prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)

batch_size = 16
prediction_data = tensor_data(prediction_inputs, prediction_masks)
prediction_sampler = sampler(prediction_data)
prediction_dataloader = loader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

# load the model from disk



# Put model in evaluation mode
bert_model.eval()
# Tracking variables 
predictions = []
# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = bert_model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)
  logits = outputs[0]
  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  #label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  #true_labels.append(label_ids)
print('emotions.')
import torch
import torch.nn.functional as F 
#caluclate the softmax for logits from bert model
#becasue we have 7 different calsses otherwise we use sigmoid for binary classification
softmax_val=F.softmax(torch.tensor(predictions[0])).tolist()
#print(softmax_val)
#sorting the list of softmax values to get max 2 probabilities
sorted_integers = sorted(softmax_val, reverse=True)  

largest_prob = sorted_integers[0]  
second_largest_prob = sorted_integers[1]  

#Here we are extracting the max probability value index from the list and get the key(label) from label dictionary

#max_value = max((softmax_val))
max_index1 = softmax_val.index(largest_prob)
max_index2 = softmax_val.index(second_largest_prob)
key_list = list(label_dict.keys()) 
val_list = list(label_dict.values()) 
  
emotion1=(key_list[val_list.index(max_index1)]) 
 
print(emotion1)
